# Descrição experimentos usando NETNS (isolamento de interfaces usando ambientes de rede virtualizados - namespaces)

- Objetivo configurar um script para automatizar tudo.

- Ref ( muito bom inclusive ):  https://www.youtube.com/watch?v=jhk7y7mYI6s&ab_channel=TechUpskill

# Criar um namespace para cada domínio.

<!-- sudo ip link add <nome-interface> type dummy # isso nao funcionou direito, infelizmente -->

* listar namespaces ativos:
    `sudo ip netns list`

* Criar namespace:
    `sudo ip netns add VRF1`    

* Criando interfaces para utilizar com os namespaces
    `sudo ip link add veth1 type veth peer name veth2`


* Mover interface para namespace:
    `sudo ip link set veth1 netns VRF1`


* Executar comando a partir da namespace (ex: ping, /bin/bash, ifconfig):
    `sudo ip netns exec VRF1 <comando>`

* Subir as interfaces dentro dos namespaces:
    `sudo ip netns exec VRF1 ifconfig veth1 up`

* Criar ovs bridge:
    `sudo ovs-vsctl add-br switch1`

* Adicionar as interfaces criadas:
    `sudo ovs-vsctl add-port switch1 veth2`

* Se não conseguir ver as interfaces, ver com:
    `sudo ip link`

* Associar ips para as interfaces:
    `sudo ifconfig veth1 10.0.0.5/24`

* Rodar terminal a partir do namespace:
    `sudo ip netns exec VRF1 /bin/bash`

* Ou melhor ainda (igual no mininet):
    `sudo ip netns exec VRF1 xterm`

* Ver largura de banda do enlace (interface):
    `tc qdisk 